# Comparing OLS and Double Selection
This material is originally developed for ACE 592 Big Data - Empirical Economics written on 7/21/2017 by Alex Stevens. 

### Motivation
Researchers do not know the data generating process for the data they are analyzing. This exercise creates a nonlinear data generating process that is too complex for simple modelling to capture, then compares OLS and Double Selection estimation.

### Objective:
* Create Monte Carlo experiement
* Compare distribution of coeffiecients estimated by OLS and Double Selection 
* Look at the effects of assumptions on these two models

Remember to shut down your server when you are done by clicking Control Panel -> Shut Down Server

## Step 1: Install packages 

In [1]:
rm(list=ls())
## This function will check if a package is installed, and if not, install it
pkgTest <- function(x) {
  if (!require(x, character.only = TRUE))
  {
    install.packages(x, dep = TRUE, repos = "http://cran.us.r-project.org")
    if(!require(x, character.only = TRUE)) stop("Package not found")
  }
}

## These lines load the required packages
packages <- c("Hmisc", "hdm","mvtnorm")
lapply(packages, pkgTest)

Loading required package: Hmisc
Loading required package: lattice
Loading required package: survival
Loading required package: Formula
Loading required package: ggplot2

Attaching package: 'Hmisc'

The following objects are masked from 'package:base':

    format.pval, units

Loading required package: hdm
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"there is no package called 'hdm'"Installing package into 'C:/Users/luoyec2/Documents/R/win-library/3.6'
(as 'lib' is unspecified)


package 'hdm' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\luoyec2\AppData\Local\Temp\Rtmp4WbNKs\downloaded_packages


Loading required package: hdm
Loading required package: mvtnorm


[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

## Step 2: Create data


Set seed for replication

In [2]:
set.seed(12345)

Set sample size

In [3]:
N <- 200

Set number of simulations

In [4]:
n.sims <- 40

Create matrix of 200 random normally distributed variables

In [5]:
x <- matrix(runif(N*50), nrow = N, ncol = 50)

Create polynomials

In [6]:
polyx<-cbind(x^2,x^3)

Create 5 bins for first 40 variables

In [7]:
nvar<-10
quants<-5
dumx<-matrix(nrow=N,ncol=quants*nvar)
  for(i in 1:nvar){
    indx<- factor(as.numeric(cut2(x[,i], g=quants)))
    dumx[,((quants*i)-(quants-1)):(quants*i)]<-model.matrix(~indx-1)
  }

Create treatment variable where half of the observations are treatment half are control

In [8]:
 Treatment<-sample(rep(0:1,N/2))

Create true y. Note:
* True effect of Treatment on y is 1
* Most x variables have no effect on y
* Some x variables have nonlinear effects on y

In [9]:
  y <- Treatment+x[,1] * 2+ x[,2] * 2.5 + 600*x[,3]-20*x[,3]^2-20*x[,3]^3+
                #3*exp(x[,4]+x[,10])+2*log(x[,6]+x[,10]+100)+ 
                600*x[,7]^2+30*x[,8]^2- 20*x[,8]^3+
                300*dumx[,(quants*5)]+90*dumx[,(quants*5)+1]+30*dumx[,(quants*5)+2]+
                100*dumx[,(quants*8)+1]+50*dumx[,(quants*8)+2]+70*dumx[,(quants*8)+3]+
                10*dumx[,(quants*9)]+60*dumx[,(quants*9)+3]-6*dumx[,(quants*9)+5]+
                rnorm(N)

## Step 3: Analyze Data

Estimate naive OLS model using only x variables and summarize

In [10]:
fit.lm <- lm(y~Treatment+x)
summary(fit.lm)


Call:
lm(formula = y ~ Treatment + x)

Residuals:
     Min       1Q   Median       3Q      Max 
-169.283  -64.938   -5.222   63.929  275.300 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -10.9946   109.2828  -0.101  0.92000    
Treatment   -16.9660    16.3809  -1.036  0.30202    
x1          -18.0546    28.2617  -0.639  0.52392    
x2           15.0191    28.4031   0.529  0.59775    
x3          591.5983    29.6282  19.967  < 2e-16 ***
x4            3.0586    27.6139   0.111  0.91195    
x5          317.6167    31.9311   9.947  < 2e-16 ***
x6          -79.2730    29.3840  -2.698  0.00779 ** 
x7          552.3871    27.5115  20.078  < 2e-16 ***
x8          -17.5010    28.5975  -0.612  0.54149    
x9          -73.1006    30.8280  -2.371  0.01902 *  
x10          10.7842    29.6723   0.363  0.71679    
x11          35.1361    28.9508   1.214  0.22681    
x12         -57.0955    28.3547  -2.014  0.04586 *  
x13          31.9717    29.2230   1.094  0.2757

Estimate naive Double Selection model using only x variables and summarize

In [11]:
fit.ds <- rlassoEffect(x, y, Treatment, method = "double selection")
summary(fit.ds)

[1] "Estimates and significance testing of the effect of target variables"
   Estimate. Std. Error t value Pr(>|t|)  
d1    -25.48      14.63  -1.742   0.0816 .
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1



Now assume some x variables have polynomial effects on y and estimate OLS with x and polyx variables

In [12]:
fit.lm.os <- lm(y~Treatment+x+polyx)
  summary(fit.lm.os)


Call:
lm(formula = y ~ Treatment + x + polyx)

Residuals:
    Min      1Q  Median      3Q     Max 
-87.342 -25.054  -0.034  21.517 111.390 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)  
(Intercept)   117.156    283.118   0.414   0.6809  
Treatment     -14.026     19.460  -0.721   0.4745  
x1            -14.298    303.583  -0.047   0.9626  
x2           -220.140    397.343  -0.554   0.5821  
x3            801.049    316.597   2.530   0.0147 *
x4           -133.648    350.983  -0.381   0.7050  
x5            252.258    459.715   0.549   0.5857  
x6           -239.628    295.502  -0.811   0.4214  
x7             -7.961    372.981  -0.021   0.9831  
x8            387.377    385.801   1.004   0.3204  
x9             97.009    288.737   0.336   0.7384  
x10           249.887    369.919   0.676   0.5026  
x11           374.552    310.845   1.205   0.2341  
x12           113.206    426.583   0.265   0.7919  
x13            58.257    315.895   0.184   0.8545  
x14         

Assume some x variables have binned effects on y and estimate OLS 

In [13]:
fit.lm.os <- lm(y~Treatment+x+dumx+polyx)
  summary(fit.lm.os)


Call:
lm(formula = y ~ Treatment + x + dumx + polyx)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.48316 -0.10450  0.01509  0.10135  0.45388 

Coefficients: (10 not defined because of singularities)
              Estimate Std. Error  t value Pr(>|t|)    
(Intercept)  264.55350   22.02832   12.010 2.13e-06 ***
Treatment      1.54743    0.53805    2.876 0.020638 *  
x1            27.58123   17.99227    1.533 0.163829    
x2             1.33839   16.56898    0.081 0.937603    
x3           617.96279   14.44433   42.782 9.82e-11 ***
x4            27.94150   29.85277    0.936 0.376675    
x5           -39.71809   20.04958   -1.981 0.082921 .  
x6             6.45897   14.56738    0.443 0.669233    
x7            -9.66039   21.17964   -0.456 0.660433    
x8             0.09219   12.84239    0.007 0.994448    
x9            50.43060   21.41125    2.355 0.046294 *  
x10           22.22754   17.30348    1.285 0.234888    
x11          -19.72608   14.82110   -1.331 0.219888    
x

Assume some x variables have binned and polynomial effects on y and estimate double selection

In [14]:
xds<-cbind(x,dumx,polyx)
  
fit.ds.os <- rlassoEffect(xds, y, Treatment, method = "double selection")
summary(fit.ds.os)

[1] "Estimates and significance testing of the effect of target variables"
   Estimate. Std. Error t value Pr(>|t|)  
d1    0.6025     0.3404    1.77   0.0767 .
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1



# Step 4: Monte Carlo

Each of these method give much different coefficients for Treatment. We will use a Monte Carlo to visualize the distribution of coefficients and measure the type 1 and type 2 errors for the tests.

### Create output matrices
Prefixes:
* betahat~ -- estimates of Treatment coefficient
* type1~ -- type one error(1), no type one error(0)
* type2~ -- type two error(1), no type two error(0), where false null is less than or equal to zero 

Suffixes:
* ~ols -- OLS estimation naive model
* ~ols.os -- OLS estimation overspecified model
* ~ds -- DS estimation naive model
* ~ds.os -- DS estimation overspecified model

In [15]:
betahat.ols <- matrix(NA,nrow=n.sims,ncol=3)
betahat.ds <- matrix(NA,nrow=n.sims,ncol=1)
betahat.ols.os <- matrix(NA,nrow=n.sims,ncol=3)
betahat.ds.os <- matrix(NA,nrow=n.sims,ncol=1)
type1.ols <- matrix(NA,nrow=n.sims,ncol=1)
type1.ds <- matrix(NA,nrow=n.sims,ncol=1)
type1.ols.os <- matrix(NA,nrow=n.sims,ncol=1)
type1.ds.os <- matrix(NA,nrow=n.sims,ncol=1)
type2.ols <- matrix(NA,nrow=n.sims,ncol=1)
type2.ds <- matrix(NA,nrow=n.sims,ncol=1)
type2.ols.os <- matrix(NA,nrow=n.sims,ncol=1)
type2.ds.os <- matrix(NA,nrow=n.sims,ncol=1)


##### Takes 15-20 minutes to run
* Run the Monte Carlo experiment using previously described data generating process

* Calculate type 1 and type 2 errors

In [16]:

for(j in 1:n.sims){
# Create data  
  x <- matrix(runif(N*50), nrow = N, ncol = 50)
  Treatment<-sample(rep(0:1,N/2))
  polyx<-cbind(x^2,x^3)
  nvar<-20
    quants<-5
    dumx<-matrix(nrow=N,ncol=5*nvar)
  for(i in 1:nvar){
    indx<- factor(as.numeric(cut2(x[,i], g=quants)))
    dumx[,((quants*i)-(quants-1)):(quants*i)]<-model.matrix(~indx-1)
  }
# Create y    
  y <- Treatment+x[,1] * 2+ x[,2] * 2.5 + 600*x[,3]-20*x[,3]^2-20*x[,3]^3+
                #3*exp(x[,4]+x[,10])+2*log(x[,6]+x[,10]+100)+ 
                600*x[,7]^2+30*x[,8]^2- 20*x[,8]^3+
                300*dumx[,(quants*5)]+90*dumx[,(quants*5)+1]+30*dumx[,(quants*5)+2]+
                100*dumx[,(quants*8)+1]+50*dumx[,(quants*8)+2]+70*dumx[,(quants*8)+3]+
                10*dumx[,(quants*9)]+60*dumx[,(quants*9)+3]-6*dumx[,(quants*9)+5]+
                rnorm(N)
#### Estimating true relationships
# Estimate naive OLS  
  fit.lm <- lm(y~Treatment+x)
  summary(fit.lm)
  
  b<-coef(summary(fit.lm))[,"Estimate"][2]
  s<-coef(summary(fit.lm))[,"Std. Error"][2]
  p<-coef(summary(fit.lm))[,"Pr(>|t|)"][2]
   
  t<-(b-1)/(s/sqrt(N))
# Store relavant statistics
  betahat.ols[j,]<-b
  
  if(t>1.96){
    type1.ols[j,1]<-1
  }
  else{
    type1.ols[j,1]<-0
  }
  
  if(b<0 | p>.05){
    type2.ols[j,1]<-1
  }
  else{
    type2.ols[j,1]<-0
  }

# Estimate naive DS  
  fit.ds <- rlassoEffect(x, y, Treatment, method = "double selection")
  summary(fit.ds)
  
  b.ds<-coef(summary(fit.ds))[,"Estimate."][1]
  s.ds<-coef(summary(fit.ds))[,"Std. Error"][1]
  p.ds<-coef(summary(fit.ds))[,"Pr(>|t|)"][1]
  
  t.ds<-(b.ds-1)/(s.ds/sqrt(N))
# Store relavant statistics  
  betahat.ds[j,]<-b.ds
  
  if(t.ds>1.96){
    type1.ds[j,1]<-1
  }
  else{
    type1.ds[j,1]<-0
  }
  if(b.ds<0 | p.ds>.05){
    type2.ds[j,1]<-1
  }
  else{
    type2.ds[j,1]<-0
  }
 
## Overspecify
# Estimate overspecified OLS 
  fit.lm.os <- lm(y~Treatment+x+dumx)
  summary(fit.lm.os)
  
  
  b.os<-coef(summary(fit.lm.os))[,"Estimate"][2]
  s.os<-coef(summary(fit.lm.os))[,"Std. Error"][2]
  p.os<-coef(summary(fit.lm.os))[,"Pr(>|t|)"][2]
  
  t.os<-(b.os-1)/(s.os/sqrt(N))
# Store relavant statistics  
  betahat.ols.os[j,]<-b.os
  
  if(t.os>1.96){
    type1.ols.os[j,1]<-1
  }
  else{
    type1.ols.os[j,1]<-0
  }
  
  if(b.os<0 | p.os>.05){
    type2.ols.os[j,1]<-1
  }
  else{
    type2.ols.os[j,1]<-0
  }

# Estimate overspecified DS
  xds<-cbind(x,dumx,polyx)
  
  fit.ds.os <- rlassoEffect(xds, y, Treatment, method = "double selection")
  summary(fit.ds.os)
  
  b.ds.os<-coef(summary(fit.ds.os))[,"Estimate."][1]
  s.ds.os<-coef(summary(fit.ds.os))[,"Std. Error"][1]
  p.ds.os<-coef(summary(fit.ds.os))[,"Pr(>|t|)"][1]
  
  t.ds.os<-(b.ds.os-1)/(s.ds.os/sqrt(N))
# Store relavant statistics  
  betahat.ds.os[j,]<-b.ds.os
  
  if(t.ds.os>1.96){
    type1.ds.os[j,1]<-1
  }
  else{
    type1.ds.os[j,1]<-0
  }
  
  if(b.ds.os<0 | p.ds.os>.05){
    type2.ds.os[j,1]<-1
  }
  else{
    type2.ds.os[j,1]<-0
  }
  print(paste0(j, ' of ',n.sims))
}

# Print type 1 and type 2 errors
print(paste0('type 1 error naive OLS rate = ',colMeans(type1.ols)))
print(paste0('type 1 error naive DS rate = ',colMeans(type1.ds)))
print(paste0('type 1 error overspecified OLS rate = ',colMeans(type1.ols.os)))
print(paste0('type 1 error overspecified DS rate = ',colMeans(type1.ds.os)))

print(paste0('type 2 error naive OLS rate = ',colMeans(type2.ols)))
print(paste0('type 2 error naive DS rate = ',colMeans(type2.ds)))
print(paste0('type 2 error overspecified OLS rate = ',colMeans(type2.ols.os)))
print(paste0('type 2 error overspecified DS rate = ',colMeans(type2.ds.os)))



[1] "1 of 40"
[1] "2 of 40"
[1] "3 of 40"
[1] "4 of 40"
[1] "5 of 40"
[1] "6 of 40"
[1] "7 of 40"
[1] "8 of 40"
[1] "9 of 40"
[1] "10 of 40"
[1] "11 of 40"
[1] "12 of 40"
[1] "13 of 40"
[1] "14 of 40"
[1] "15 of 40"
[1] "16 of 40"
[1] "17 of 40"
[1] "18 of 40"
[1] "19 of 40"
[1] "20 of 40"
[1] "21 of 40"
[1] "22 of 40"
[1] "23 of 40"
[1] "24 of 40"
[1] "25 of 40"
[1] "26 of 40"
[1] "27 of 40"
[1] "28 of 40"
[1] "29 of 40"
[1] "30 of 40"
[1] "31 of 40"
[1] "32 of 40"
[1] "33 of 40"
[1] "34 of 40"
[1] "35 of 40"
[1] "36 of 40"
[1] "37 of 40"
[1] "38 of 40"
[1] "39 of 40"
[1] "40 of 40"
[1] "type 1 error naive OLS rate = 0.425"
[1] "type 1 error naive DS rate = 0.45"
[1] "type 1 error overspecified OLS rate = 0.45"
[1] "type 1 error overspecified DS rate = 0.45"
[1] "type 2 error naive OLS rate = 0.9"
[1] "type 2 error naive DS rate = 0.875"
[1] "type 2 error overspecified OLS rate = 0.975"
[1] "type 2 error overspecified DS rate = 0.4"


Save density plots of coefficients

In [17]:
png(filename=paste(file.path(path.expand('~'),'outputs/Lab6/'),'MisspecDS.png'))

plot (density(betahat.ds),col="red",main = "",xlab="Treatment Coefficient Estimate (True = 1)")
title(main = "Density Plot of naive Double Selection")

dev.off()

png(filename=paste(file.path(path.expand('~'),'outputs/Lab6/'),'MisspecOLS.png'))

plot (density(betahat.ols),col="red",main = "",xlab="Treatment Coefficient Estimate (True = 1)")
title(main = "Density Plot of naive OLS")

dev.off()


png(filename=paste(file.path(path.expand('~'),'outputs/Lab6/'),'MisspecDSos.png'))

plot (density(betahat.ds.os),col="red",main = "",xlab="Treatment Coefficient Estimate (True = 1)")
title(main = "Density Plot of overspecified Double Selection")

dev.off()

png(filename=paste(file.path(path.expand('~'),'outputs/Lab6/'),'MisspecOLSos.png'))

plot (density(betahat.ols.os),col="red",main = "",xlab="Treatment Coefficient Estimate (True = 1)")
title(main = "Density Plot of overspecified OLS")

dev.off()

Warning message in png(filename = paste(file.path(path.expand("~"), "outputs/Lab6/"), :
"unable to open file 'C:\Users\luoyec2\Documents/outputs/Lab6 MisspecDS.png' for writing"Warning message in png(filename = paste(file.path(path.expand("~"), "outputs/Lab6/"), :
"opening device failed"

ERROR: Error in png(filename = paste(file.path(path.expand("~"), "outputs/Lab6/"), : unable to start png() device


### Question 1

Which the four methods has the best distribution? Why? What assumptions of OLS and DS are violated in the naive models? 

**Overspecified DS has the best performance. LASSO algorithm can shrink the variables; therefore the overspecification will not harm the LASSO selection procedure. The double selection algorithm can keep the most important variables in the post-selection procedure. **

### Question 2

Why does the overspecified OLS have a lower type 1 error rate? 

**Type 1 error is the error of rejecting a true null hypothesis. The overspecification OLS includes as many variables as possible after shrinkage procedure. **

### Question 3

The type 2 error rate is calculated incorrectly. Describe how would you correct it? 

**Type 2 error is the error of non-rejection of a false null hypothesis. **

### Question 4

Create a correlation between the Treatment and the independent variables. How does this change the OLS and DS coefficents? What other methods can be used to deal with this specification?

**DS can adjust that by two-stage method.**

### Question 5

Change the model or data generating process so that an assumption of double selection is violated. How does this affect the ability of the model to draw valid inference?